# TASK 1

### Import libraries

In [3]:
# univariate lstm example
from numpy import array

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

### Generate Dataset By Spliting The Input List Into Sequece Of Data :
    * Create two sets of lists 
    * Split the input into steps and store in variable X
    * Store the last of each of these variables into y
    * Stop if the steps are greater than the lenght of the input data

In [4]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

### import Accuracy metric

In [5]:
import tensorflow as tf

#### Split The Data into X and Y and Reshape the X

In [6]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
n_steps = 3
X, y = split_sequence(raw_seq, n_steps)

n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))


### Build Model 
    * LSTM layer
    * Dense layer 
   
   
#### Show summary of the model layers

In [7]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


2021-08-15 15:15:13.153358: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-15 15:15:13.153401: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-15 15:15:13.153438: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (olamide): /proc/driver/nvidia/version does not exist
2021-08-15 15:15:13.153845: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Train the model on the Input X,y and predict on a test data 

In [8]:
model.fit(X, y, epochs=200, verbose=0)
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[103.03189]]


#### Change Values Of Epochs To 3

In [27]:
model.fit(X, y, epochs=3, verbose=0)
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
# predict Next input
print(yhat)

[[101.92971]]


### Define accuracy 

In [28]:
def accuracy(actual, predicted):
    deviation = (actual-predicted)/actual
    percentage = 100 - (deviation * 100)
    return percentage

In [29]:
print(f'percentage after 3 epochs : {accuracy(yhat[0], array(100))}%')

percentage after 3 epochs : [98.10682]%


In [30]:
model.fit(X, y, epochs=5, verbose=0)
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[101.9002]]


In [34]:
print(f'percentage after 5 epochs : {accuracy(yhat[0], array(100))[0]}%')

percentage after 5 epochs : 98.1915512084961%


In [42]:
model.fit(X, y, epochs=10, verbose=0)
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[100.43257]]


In [43]:
print(f'percentage after 10 epochs : {accuracy(yhat[0], array(100))[0]}%')

percentage after 10 epochs : 99.56929016113281%


### Predict The Next Two Values

In [40]:
model.fit(X, y, epochs=200, verbose=0)
x_input = array(([70, 80, 90], [80,90,100]))
x_input = x_input.reshape((x_input.shape[0], x_input.shape[1], n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[100.432076]
 [111.242516]]


# TASK 2

# DML Workshop 06 - Text Generation using LSTM

# Import Packages

In [1]:
# Load LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

2022-01-15 17:33:08.703621: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-15 17:33:08.703664: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Data Preprocessing
    Open word file and load text into memory
    convert the text loaded into lowercase
    sort the texts and remove recurring letters

In [2]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))


### Data Preparation
    1. Map the unique characters to numbers and the unique numbers to characters
    2. Now that the letters have been represented by numbers for training
    3. The words are sliced into diffrent group (sentences) for Training
    4. The next word after this is represented as the predicted variable
    5. Then coverted to numbers , and seperated into Taarget and Input variables



In [3]:
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)
# define the LSTM model


Total Characters:  148574
Total Vocab:  46
Total Patterns:  148474


## Model 
    Define a sequential model 
    then add an lstm layer to learn the important words
    Add Dropout to compensate overfitting

In [5]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

2022-01-15 17:34:06.360306: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-15 17:34:06.360339: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-15 17:34:06.360363: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (olamide): /proc/driver/nvidia/version does not exist
2022-01-15 17:34:06.360733: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Save Model And Store Checkpoints (Where to pick up model training)


In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=5, batch_size=128, callbacks=callbacks_list)
    

Epoch 1/5
1160/1160 [==============================] - ETA: 0s - loss: 2.9446
Epoch 00001: loss improved from inf to 2.94464, saving model to weights-improvement-01-2.9446.hdf5
1160/1160 [==============================] - 422s 362ms/step - loss: 2.9446
Epoch 2/5
1160/1160 [==============================] - ETA: 0s - loss: 2.7038
Epoch 00002: loss improved from 2.94464 to 2.70384, saving model to weights-improvement-02-2.7038.hdf5
1160/1160 [==============================] - 410s 353ms/step - loss: 2.7038
Epoch 3/5
1160/1160 [==============================] - ETA: 0s - loss: 2.6012
Epoch 00003: loss improved from 2.70384 to 2.60120, saving model to weights-improvement-03-2.6012.hdf5
1160/1160 [==============================] - 402s 346ms/step - loss: 2.6012
Epoch 4/5
1160/1160 [==============================] - ETA: 0s - loss: 2.5337
Epoch 00004: loss improved from 2.60120 to 2.53374, saving model to weights-improvement-04-2.5337.hdf5
1160/1160 [==============================] - 407s 35

### Generate The Text Using The Model

In [6]:
# load the network weights
filename = "weights-improvement-05-2.4748.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
print("Generating Text")
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"  and washing--extra."'

  `you couldn't have wanted it much,' said alice; `living at the
bottom of t "
Generating Text
he woeee                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

### Changing Number Of Epochs to 25
    Checking the Accuracy
    Accuracy Improves till 25 epochs 

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# define the checkpoint
filepath="weights-improfvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=25, batch_size=128, callbacks=callbacks_list)
    

Epoch 1/25
1160/1160 [==============================] - ETA: 0s - loss: 2.4197 - accuracy: 0.3160
Epoch 00001: loss improved from inf to 2.41969, saving model to weights-improfvement-01-2.4197.hdf5
1160/1160 [==============================] - 385s 331ms/step - loss: 2.4197 - accuracy: 0.3160
Epoch 2/25
1160/1160 [==============================] - ETA: 0s - loss: 2.3770 - accuracy: 0.3271
Epoch 00002: loss improved from 2.41969 to 2.37696, saving model to weights-improfvement-02-2.3770.hdf5
1160/1160 [==============================] - 388s 334ms/step - loss: 2.3770 - accuracy: 0.3271
Epoch 3/25
1160/1160 [==============================] - ETA: 0s - loss: 2.3336 - accuracy: 0.3376
Epoch 00003: loss improved from 2.37696 to 2.33356, saving model to weights-improfvement-03-2.3336.hdf5
1160/1160 [==============================] - 391s 337ms/step - loss: 2.3336 - accuracy: 0.3376
Epoch 4/25
1160/1160 [==============================] - ETA: 0s - loss: 2.2961 - accuracy: 0.3468
Epoch 00004: lo

In [7]:
# load the network weights
filename = "weights-improfvement-25-1.7584.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
print("Generating Text")
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" he was close behind it when she turned the
corner, but the rabbit was no longer to be seen:  she fou "
Generating Text
nd the fad not to the tooe     `whu did tot taid to toee to toye
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

### Changing Numbers Of Epochs to 50
    Accuracy improves till 41 epochs 
    no changes in accuracy after 
    Texts are meaningless 

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# define the checkpoint
filepath="weights-improvemffent-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)
    

Epoch 1/50
1160/1160 [==============================] - ETA: 0s - loss: 2.9399 - accuracy: 0.2016
Epoch 00001: loss improved from inf to 2.93987, saving model to weights-improvemffent-01-2.9399.hdf5
1160/1160 [==============================] - 343s 295ms/step - loss: 2.9399 - accuracy: 0.2016
Epoch 2/50
1160/1160 [==============================] - ETA: 0s - loss: 2.7040 - accuracy: 0.2522
Epoch 00002: loss improved from 2.93987 to 2.70396, saving model to weights-improvemffent-02-2.7040.hdf5
1160/1160 [==============================] - 352s 304ms/step - loss: 2.7040 - accuracy: 0.2522
Epoch 3/50
1160/1160 [==============================] - ETA: 0s - loss: 2.6088 - accuracy: 0.2740
Epoch 00003: loss improved from 2.70396 to 2.60876, saving model to weights-improvemffent-03-2.6088.hdf5
1160/1160 [==============================] - 356s 307ms/step - loss: 2.6088 - accuracy: 0.2740
Epoch 4/50
1160/1160 [==============================] - ETA: 0s - loss: 2.5402 - accuracy: 0.2873
Epoch 00004:

Epoch 29/50
1160/1160 [==============================] - ETA: 0s - loss: 1.7522 - accuracy: 0.4859
Epoch 00029: loss improved from 1.76418 to 1.75215, saving model to weights-improvemffent-29-1.7522.hdf5
1160/1160 [==============================] - 350s 302ms/step - loss: 1.7522 - accuracy: 0.4859
Epoch 30/50
1160/1160 [==============================] - ETA: 0s - loss: 1.7373 - accuracy: 0.4888
Epoch 00030: loss improved from 1.75215 to 1.73734, saving model to weights-improvemffent-30-1.7373.hdf5
1160/1160 [==============================] - 350s 302ms/step - loss: 1.7373 - accuracy: 0.4888
Epoch 31/50
1160/1160 [==============================] - ETA: 0s - loss: 1.7280 - accuracy: 0.4904
Epoch 00031: loss improved from 1.73734 to 1.72803, saving model to weights-improvemffent-31-1.7280.hdf5
1160/1160 [==============================] - 351s 302ms/step - loss: 1.7280 - accuracy: 0.4904
Epoch 32/50
1160/1160 [==============================] - ETA: 0s - loss: 1.7134 - accuracy: 0.4940
Epoc

In [8]:
# load the network weights
filename = "weights-improvemffent-41-1.6294.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
print("Generating Text")
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" oesn't matter much,' thought alice, `as all the arches
are gone from this side of the ground.'  so s "
Generating Text
he went on, `i mort titel at the ragtin in an in oittle horeer the rail of the war oo the taye aerer at the oadb (she in the tam   she mad shat the was setiing toeerle, and she celln it tae tut thin she was noteing toee theer the rage ou teye oi the court, and the had to line the rabe turt alrcer to the kand.

                 *                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

### Stacking LSTM layer 

In [9]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# define the checkpoint
filepath="weights-improevemffent-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=10, batch_size=128, callbacks=callbacks_list)
    

Epoch 1/10
1160/1160 [==============================] - ETA: 0s - loss: 3.0217 - accuracy: 0.1940
Epoch 00001: loss improved from inf to 3.02171, saving model to weights-improevemffent-01-3.0217.hdf5
1160/1160 [==============================] - 1530s 1s/step - loss: 3.0217 - accuracy: 0.1940
Epoch 2/10
1160/1160 [==============================] - ETA: 0s - loss: 3.0176 - accuracy: 0.1945
Epoch 00002: loss improved from 3.02171 to 3.01760, saving model to weights-improevemffent-02-3.0176.hdf5
1160/1160 [==============================] - 1549s 1s/step - loss: 3.0176 - accuracy: 0.1945
Epoch 3/10
1160/1160 [==============================] - ETA: 0s - loss: 3.0145 - accuracy: 0.1945
Epoch 00003: loss improved from 3.01760 to 3.01449, saving model to weights-improevemffent-03-3.0145.hdf5
1160/1160 [==============================] - 1561s 1s/step - loss: 3.0145 - accuracy: 0.1945
Epoch 4/10
1160/1160 [==============================] - ETA: 0s - loss: 3.0144 - accuracy: 0.1945
Epoch 00004: lo

#### Generated Texts Makes more Sense 
LSTM learns the important information from the training data and generates a more meaningful text 


In [10]:
# load the network weights
filename = "weights-improevemffent-04-3.0144.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
print("Generating Text")
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"  man, your majesty,' the hatter began, in a
trembling voice, `--and i hadn't begun my tea--not above "
Generating Text
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         